In [1]:
import torch.nn as nn
import torch
from torch.autograd import Variable

In [2]:
class Controller(nn.Module):
    """
    Contains the two learnable parts of the model in four independent, fully connected layers.
    First the initial values for the registers and instruction registers and second the 
    parameters that computes the required distributions. 
    """
    def __init(self, program_matrix1, program_matrix2, program_matrix3, program_matrix4, initial_memory, 
               initial_registers, instruction_register, stop_threshold):
        """
        Initializes registers, memory and register matrix and their dimensions 
        Initializes four program matrices one of dimension NxM and three of dimension RxM
        """
        super(Controller, self).__init__()
        R, M = initial_registers.size()
        self.M = M
        self.R = R
        
        self.program_matrix1 = nn.Parameter(program_matrix1)
        self.program_matrix2 = nn.Parameter(program_matrix2)
        self.program_matrix3 = nn.Parameter(program_matrix3)
        self.program_matrix4 = nn.Parameter(program_matrix4)    
        
        # Memory matrix (M x M)
        self.memory = initial_memory
        
        # Register Matrix (R x M)
        self.registers = initial_registers
        
        # Instruction Register (M)
        self.IR = instruction_register
        
        # Machine initialization
        self.Machine = Machine(M, R, stop_threshold)
                
    def forward(self):
        einput = torch.matmul(self.program_matrix1, self.IR)
        ainstruction = torch.matmul(self.program_matrix2, self.IR)
        binstruction = torch.matmul(self.program_matrix3, self.IR)
        outputin = torch.matmul(self.program_matrix4, self.IR)
        
        # Updating memory, registers, and IR after machine operation
        self.memory, self.registers, self.IR = self.Machine
        (einput, ainstruction, binstruction, outputin, memory, registers, IR)
                
        
    def lossfunctions(self, cmatrix, tjmatrix):
        """ compute four diferent loss functions and return a weighted average of the four measuring correctness, 
        halting, efficiency, and confidence"""
        sum ((cmatrix)*(self.program_matrix4 - self.memory)^2)
        ### need to add loss functions::: mehdi需要帮我们
        救命
        
        
# HOW TO BLUR? ==> Add a constant, then softmax
#TODO: Add in some fuzz factor????
      
        
        

In [3]:
class Machine(nn.Module):
    """
    The Machine executes assembly instructions passed to it by the Controller.
    It updates the given memory, registers, and instruction pointer.
    The Machine doesn't have any learnable parameters.
    """
    def __init__(self, M, R, stop_threshold):
        """
        Initializes dimensions, operations, and counters
        
        :param M: Memory length.  Integer values also take on values 0-M-1.  M is also the program length.
        :param R: Number of registers
        :stop_threshold: Accumulated probability of stopping after which the program terminates.
        """
        super(Machine, self).__init__()
        
        # Store parameters as class variables
        self.R = R
        self.M = M # Memory length (also largest number)
        self.stop_threshold = stop_threshold
        
        # Start off with 0 probability of stopping
        self.stop_probability = 0
        
        # A list of all our possible ops
        N = 
        self.ops = [
            Jump(M), 
            Stop(M), 
            Write(M), 
            Read(M), 
            Add(M), 
            Subtract(M), 
            Increment(M),
            Decrement(M),
            Min(M),
            Max(M),
            Zero(M)
        ]
#         self.ops = [
            
#         ]
        
        # Number of instructions
        self.N = len(self.ops)
        
        # Keep track of the index of certain ops which are dealt with specially
        self.jump_index = 0
        self.stop_index = 1
        self.write_index = 2
        self.read_index = 3
        

        
    def forward(self, e, a, b, o, memory, registers, IR):
        """
        Run the Machine for one timestep (corresponding to the execution of one line of Assembly).
        The first four parameter names correspond to the vector names used in the original ANC paper
        
        :param e: Probability distribution over the instruction being executed (length M)
        :param a: Probability distribution over the first argument register (length R)
        :param b: Probability distribution over the second argument register (length R)
        :param o: Probability distribution over the first argument register (length R)
        :param memory: Memory matrix (size M x M)
        :param registers: Register matrix (size R x M)
        :param IR: Instruction Register (length M)
        
        :return: The memory, registers, and instruction register after the timestep
        """
        
        # Calculate distributions over the two argument values by multiplying each 
        # register by the probability that register is being used.
        arg1 = torch.matmul(a, registers)
        arg2 = torch.matmul(b, registers)
        
        # Would be kinda nice to not have to for-loop this
        # Or maybe do this once
        # CUUUUUDAAAA
        
        # N x M Matrix of output values.  The i,j cell represents the probability
        # of getting value j by running instruction i.
        out = Variable(torch.zeros(self.N, self.M))
        
#         # Loop through all possible instructions
#         for k in range(self.N):
#             # Loop through all possible first arguments
#             for i in range(self.M):
#                 # Loop through all possible second arguments
#                 for j in range(self.M):
#                     if k == self.read_index:
                        
#                         mem_val = memory[i]
#                         # Loop through distribution of memory values, throw these into the matrix.
#                         for int_val in range(mem_val.size()[0]):
#                             out[k, int_val] = out[k, int_val] + arg1[i] * arg2[j] * mem_val[int_val].data[0]
#                     else:
#                         val = self.ops[k](i,j)
#                         # Store output in the appropriate matrix cell.
#                         out[k, val] = out[k, val] + arg1[i] * arg2[j]

        out_vec = torch.zeros(1, N)
        for op_index in range(self.N):
#             out_vec = out_vec + e[op_index] * torch.matmul(torch.matmul(torch.t(arg1), self.ops[op_index]), arg2)
            out_vec = torch.matmul(torch.t(e), ops) 
            
        
            
#         # Take a weighted sum over the output distributions from each instruction
#         out_vec = torch.matmul(e, out)
        
#         # Update our memory, registers, instruction register, and stopping probability
#         memory = self.writeMemory(e, memory, arg1, arg2)
#         registers = self.writeRegisters(out_vec, o, registers)
#         IR = self.updateIR(e, IR, arg1, arg2)
#         should_stop = self.updateStop(e)
        
        return(memory, registers, IR)
        
        
    def writeRegisters(self, out, o, registers):
        """
        Write the result of our operation to our registers.
        
        :param out: probability distribution over the output value
        :param o: probability distribution over the output register
        :param registers: register matrix
        
        :return: the updated registers
        """
        # Multiply probability of writing to each output register by the value 
        new_register_vals = torch.matmul(torch.unsqueeze(o,1), torch.unsqueeze(out,0))
        # Multiply each original register cell by the probabilty of not writing to that register
        old_register_vals = torch.unsqueeze((1-o),1).expand(self.R, self.M) * registers
        
        registers =  new_register_vals + old_register_vals
        return registers
    
    def updateIR(self, e, IR, arg1, arg2):
        """
        Update the instruction register
        
        :param e: distribution over the current instruction (length N)
        :param IR: instruction register (length M)
        :param arg1: distribution over the first argument value (length M)
        :param arg2: distribution over the second argument value (length M)
        
        :return: the updated instruction register
        """
        # IR - length M vector
        jump_probability = e[self.jump_index]
        is_zero = arg1[0]
        # If we're not jumping, just shift IR by one slot
        IR_no_jump = torch.cat([IR[-1], IR[:-1]])
        # If we are on a jump instruction, check whether the argument's 0.
        # If it is, jump to the location specified by arg2.  Otherwise, increment like normal.
        IR_jump = arg2 * is_zero + (1 - is_zero) * IR_no_jump
        
        IR = IR_no_jump * (1 - jump_probability) + IR_jump * jump_probability
        return IR
    
    def writeMemory(self, e, mem_orig, arg1, arg2):
        """
        Update the memory
        
        :param e: distribution over the current instruction (length M)
        :param mem_orig: current memory matrix (size M x M)
        :param arg1: distribution over the first argument value (length M)
        :param arg2: distribution over the second argument value (length M)
        
        :return: the updated memory matrix
        """
        write_probability = e[self.write_index]
        # If we are on a write instruction, write the value arg2 in register arg1. Otherwise, leave memory as is.
        mem_write = torch.matmul(torch.unsqueeze(arg1,1), torch.unsqueeze(arg2,0))        
        memory = mem_orig * (1 - write_probability) + mem_write * write_probability
        return memory
        
    def updateStop(self, e):
        """
        Update the probability that we will stop at this timestep based on the probability that we are running the STOP op.
        
        :param e: distribution over the current instruction (length M)
        
        :return: boolean representing whether the controller should stop.
        """
        self.stop_probability += e[self.stop_index].data[0]
        if self.stop_probability > self.stop_threshold:
            return True #should stop
        return False #shouldn't stop


In [4]:
class Operation(nn.Module):
    """
    Parent class for our binary operations
    """
    def __init__(self, M):
        """
        Initialize the memory length (needed so we can mod our answer in case it exceeds the range 0-M-1)
        
        :param M: Memory length
        """
        super(Operation, self).__init__()
        self.M = M #TODO: Check this gets updated!
    
    def forward(self, x, y):
        """ 
        Perform the binary operation.  The arguments may or may not be used.
        
        :param x: First argument
        :param y: Second argument
        """
        raise NotImplementedError

In [5]:
class Add(Operation):

    def __init__(self, M):
        super(Add, self).__init__(M)
    
    def forward(self, x, y):
        return (x + y) % self.M


In [6]:
class Stop(Operation):
    
    def __init__(self, M):
        super(Stop, self).__init__(M)

    def forward(self, _1, _2):
        return 0

In [7]:
class Jump(Operation):
    
    def __init__(self, M):
        super(Jump, self).__init__(M)

    def forward(self, _1, _2):
        return 0 # Actual jump happens in the Machine class

In [8]:
class Decrement(Operation):
    
    def __init__(self, M):
        super(Decrement, self).__init__(M)

    def forward(self, x, _):
        return (x - 1) % self.M

In [9]:
class Increment(Operation):
    
    def __init__(self, M):
        super(Increment, self).__init__(M)

    def forward(self, x, _):
        return (x + 1) % self.M

In [10]:
class Max(Operation):
    
    def __init__(self, M):
        super(Max, self).__init__(M)

    def forward(self, x, y):
        return max(x,y)

In [11]:
class Min(Operation):
    
    def __init__(self, M):
        super(Min, self).__init__(M)

    def forward(self, x, y):
        return min(x,y)

In [12]:
class Read(Operation):
    
    def __init__(self, M):
        super(Read, self).__init__(M)

    def forward(self, x, _):
        return self.memory[x]

In [13]:
class Subtract(Operation):
    
    def __init__(self, M):
        super(Subtract, self).__init__(M)

    def forward(self, x, y):
        return (x - y) % self.M

In [14]:
class Write(Operation):
    
    def __init__(self, M):
        super(Write, self).__init__(M)

    def forward(self, x, y):
        return 0 # Actual write happens in the Machine class

In [15]:
class Zero(Operation):
    
    def __init__(self, M):
        super(Zero, self).__init__(M)

    def forward(self, _1, _2):
        return 0

In [16]:
# Mini-test
# M=4, R=2, N=11
machine = Machine(3, 2, .5)
e = Variable(torch.FloatTensor([.1,.1,.1,.1,.1,.1,.1,.1,.1,.05,.05]))
a = Variable(torch.FloatTensor([.1, .9]))
b = Variable(torch.FloatTensor([.8, .2]))
o = Variable(torch.FloatTensor([.6, .4]))
memory = Variable(torch.FloatTensor([[1,0,0], [1,0,0], [0,0,1]]))
registers = Variable(torch.FloatTensor([[.4, .5, .1], [.2, .6, .2]]))
IR = Variable(torch.FloatTensor([.1, .9, .1]))
mem, regs, ir = machine(e, a, b, o, memory, registers, IR)
print("MEM", mem)
print("REGS", regs)
print("IR", ir)

MEM Variable containing:
 0.9079  0.0114  0.0026
 0.9212  0.0307  0.0071
 0.0068  0.0099  0.9023
[torch.FloatTensor of size 3x3]

REGS Variable containing:
 0.5373  0.3134  0.1493
 0.3715  0.4356  0.1929
[torch.FloatTensor of size 2x3]

IR Variable containing:
 0.1057
 0.1092
 0.8828
[torch.FloatTensor of size 3]



In [17]:
# TASKS 
# - Compilation
# - Rest of the operations
# - Train function
# - Controller
# - Cuda
# - Blurring
# - Running the tests they ran, verifying that we get similar results

In [18]:
#PUT THE TRAINING FUNCTION HERE.  COPY AND PASTE A BUNCHA STUFF